In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import StandardScaler  

from sklearn.model_selection import RandomizedSearchCV
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)
properati=properati.dropna().reset_index()

<IPython.core.display.Javascript object>

In [6]:
clf = svm.SVC()
scaler = StandardScaler()  
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)

# tenia un nan nose porque
properati.dropna(inplace=True)
# lo casteo a float porque si no tira error
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 


In [42]:
properati.groupby('categories_by_price').count()

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,186,186,186,186,186,186,186,186,186,186,...,186,186,186,186,186,186,186,186,186,186
4,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
5,677,677,677,677,677,677,677,677,677,677,...,677,677,677,677,677,677,677,677,677,677
6,452,452,452,452,452,452,452,452,452,452,...,452,452,452,452,452,452,452,452,452,452
7,261,261,261,261,261,261,261,261,261,261,...,261,261,261,261,261,261,261,261,261,261
8,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,122,122,122,122,122,122


# Random Search

In [7]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

scaler.fit_transform(X,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


In [8]:
C_range = np.arange(0.001,1000,0.001)
gamma_range = np.arange(0.001,1000,0.001)

In [10]:
%%notify

from sklearn.model_selection import StratifiedShuffleSplit

param_grid ={"gamma": gamma_range, "C": C_range}
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3) #Se cuelga si lo pones
random_search = RandomizedSearchCV(clf,param_distributions=param_grid,n_iter=50,cv=5,n_jobs=-1)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(random_search.cv_results_['params'])))
score.report_single(random_search.cv_results_)

RandomizedSearchCV took 104.30 seconds for 50 candidate parameter settings.
Model with rank: 1
Mean training score: 0.519 (std: 0.006)
Mean validation score: 0.318 (std: 0.045)
Parameters: {'C': 567.73599999999999, 'gamma': 1.3559999999999999}

Model with rank: 2
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.313 (std: 0.020)
Parameters: {'C': 975.46500000000003, 'gamma': 235.00400000000002}

Model with rank: 3
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.312 (std: 0.012)
Parameters: {'C': 714.26099999999997, 'gamma': 115.313}

Model with rank: 3
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.312 (std: 0.012)
Parameters: {'C': 619.64699999999993, 'gamma': 119.015}

Model with rank: 5
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.311 (std: 0.016)
Parameters: {'C': 837.07500000000005, 'gamma': 347.435}

Model with rank: 5
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.311 (std: 0.016)
Parameters:

<IPython.core.display.Javascript object>